In [1]:
import xarray as xr
import numpy as np

# trained on >100 epochs of 60/600 (20181127-20190927)
#voodoo_nc = '20190801-20190801-20201017-174440__20201017-174440_weights.h5.nc'
voodoo_nc = '20190313-20190313-20201017-174440__20201017-174440_weights.h5.nc'


In [2]:
# open the dataset using xarray
voodoo_xr = xr.open_mfdataset(f'data/{voodoo_nc}')

T = voodoo_xr.temperature.copy()
p = voodoo_xr.pressure.copy()
T['var_unit'] = "C"
p['var_unit'] = 'hPa'
q = voodoo_xr.q.copy()

def spechum2relhum(q, T, P):
    Ttmp = T + 273.15
    T0 = 273.16  # Kelvin
    return 0.263 * P * q / np.exp(17.67 * (Ttmp - T0) / (Ttmp - 29.65))


T = T -  273.15
p = p / 100.0
q.values = spechum2relhum(q.values, T.values, p.values) * 100.0

contour_T = {'data': T, 'levels': np.arange(-40, 16, 5)}
contour_p = {'data': p, 'levels': np.arange(500, 1000, 50)}
contour_q = {'data': q, 'levels': np.arange(70, 105, 5)}

print(voodoo_xr)

<xarray.Dataset>
Dimensions:                                       (cl: 9, dt: 2882, nchannels: 6, nsamples: 188320, nvelocity: 256, rg: 292, ts: 2882)
Coordinates:
  * ts                                            (ts) float64 1.552e+09 ... 1.553e+09
  * dt                                            (dt) datetime64[ns] 2019-03-13T00:00:15.000001 ... 2019-03-14T00:00:15.000001
  * rg                                            (rg) float64 119.2 ... 1.196e+04
  * nsamples                                      (nsamples) int64 0 ... 188319
  * nvelocity                                     (nvelocity) int64 0 1 ... 255
  * nchannels                                     (nchannels) int64 0 1 2 3 4 5
  * cl                                            (cl) int64 0 1 2 3 4 5 6 7 8
Data variables:
    features                                      (nsamples, nvelocity, nchannels) float32 dask.array<shape=(188320, 256, 6), chunksize=(188320, 256, 6)>
    targets                                     

In [4]:



#voodoo_xr.features.drop(('nsamples', 'nvelocity', 'nchannels'))
#voodoo_xr.targets.drop(('nsamples', 'nvelocity', 'nchannels'))

probs = voodoo_xr.voodoo_classification_probabilities.copy()
probs['temperature'] = ( ('ts', 'rg'), voodoo_xr.temperature.copy())
probs['pressure'] = ( ('ts', 'rg'), voodoo_xr.pressure.copy())
probs['relhumidity'] = ( ('ts', 'rg'), voodoo_xr.pressure.copy())
probs['relhumidity'].values = contour_q['data']
probs['target_classification'] = ( ('ts', 'rg'), voodoo_xr.target_classification.copy())
probs['voodoo_classification'] = ( ('ts', 'rg'), voodoo_xr.voodoo_classification.copy())
probs['detection_status'] = ( ('ts', 'rg'), voodoo_xr.detection_status.copy())
probs['mask'] = ( ('ts', 'rg'), voodoo_xr.mask.copy())
probs['lwp'] = ( ('ts'), voodoo_xr.lwp.copy())
probs.to_netcdf(f"data/{voodoo_nc.replace('.h5.nc', '_compressed.h5.nc')}")

PermissionError: [Errno 13] Permission denied: b'/Users/willi/code/python/local_stuff/voodoo_quicklooks/data/20190313-20190313-20201017-174440__20201017-174440_weights_compressed.h5.nc'